In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import model_from_json
from keras.models import load_model, Sequential
from scipy import io
import models
from keras.layers import Activation, BatchNormalization, Conv2D, Conv2DTranspose, Input, concatenate, add, MaxPooling2D, UpSampling2D, Dense, Flatten

Using TensorFlow backend.


In [2]:
train_X = np.load('output_train_longi_v1.0.npy')

size = int(train_X.shape[0]/2)
train_Y = np.concatenate((np.zeros((1, size)), np.ones((1, size))), axis=1)
train_Y = keras.utils.to_categorical(train_Y, 2)

train_X = train_X.astype('float32')
train_Y = train_Y.astype('float32')

In [3]:
batch_size = 1
epochs = 40
fine_tuning = False
load_model_name = ''
epoch_to_start = 0

save_temp_model_name = 'D://Models2/temp.h'

In [4]:
model_name = 'D://Models2/classify_longi_v1.0.h5'

In [5]:
## The data, shuffled and split between train and test sets:
print('x_train shape:', train_X.shape)
print(train_X.shape[0], 'train samples')

x_train shape: (3000, 256, 256, 1)
3000 train samples


In [6]:
if fine_tuning:
    print('Loading the saved model')
    model = load_model(load_model_name)
    model.summary()

else:
    print('Start fresh')
    
    model = models.CNNClassifier(32, train_X.shape[1:])
    
    model.add(Activation('softmax'))

    opt = keras.optimizers.Adam(lr=0.01)

    model.compile(loss='mean_absolute_error', optimizer=opt)
    
    model.summary()

Start fresh
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 254, 254, 64)      18496     
_________________________________________________________________
activation_3 (Activation)    (None, 254, 254, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 252, 252, 64)      36928    

In [7]:
## Not using data augmentation.

lrScheduling = keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                 factor=0.5, patience=0, verbose=1, 
                                                 mode='auto', epsilon=0.0000001, 
                                                 cooldown=0, min_lr=0.0001)
check_point = keras.callbacks.ModelCheckpoint(save_temp_model_name, 
                                              monitor='loss', verbose=0, 
                                              save_best_only=True, save_weights_only=False, 
                                              mode='auto', period=1)
history = model.fit(train_X, train_Y,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          initial_epoch=epoch_to_start,
          callbacks = [lrScheduling, check_point])
    
np.save(model_name + '.npy', history.history)
model.save(model_name)

Epoch 1/40
 141/3000 [>.............................] - ETA: 254s - loss: 0.4574

KeyboardInterrupt: 